In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**SEARCH CONTEXT WITH WIKIPEDIA¶**

In [2]:
# installing offline dependencies
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./sentence-transformers
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126117 sha256=c032e97cf9754f5d1b38fa3544660f94c1d37e7414fd4ed4d0b4cc55a25ee4a1
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
Processing /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl
Processing /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
Processing /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
Processing /kaggle/input/llm-whls/datasets-2.14.3-py3-none-any.whl
  Attempting uninsta

In [3]:
pip uninstall tokenizers -y

Found existing installation: tokenizers 0.15.0
Uninstalling tokenizers-0.15.0:
  Successfully uninstalled tokenizers-0.15.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tokenizers==0.13.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import transformers

print("Transformers version:", transformers.__version__)


Transformers version: 4.31.0


In [6]:
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from __future__ import annotations

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")


from dataclasses import dataclass
from typing import Optional, Union

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
SIM_MODEL = '/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16

WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)
## Parameter to determine how many relevant sentences to include \ Số lượng câu liên quan
NUM_SENTENCES_INCLUDE = 25

In [49]:
# trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
# trn.drop(columns = 'id')
# trn = trn.head(4)

In [50]:
# trn

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...


**Thêm câu hỏi và lựa chọn**

In [159]:
datajson = '''
[
  {
    "prompt": "What is the Earth's primary source of energy?",
      "A": "Wind power",
      "B": "Solar radiation",
      "C": "Geothermal heat",
      "D": "Nuclear fission",
      "E": "Hydroelectric energy",
    "answer": "B"
  },
  {
    "prompt": "Which gas is most abundant in Earth's atmosphere?",
      "A": "Carbon dioxide",
      "B": "Nitrogen",
      "C": "Oxygen",
      "D": "Methane",
      "E": "Argon",
    "answer": "B"
  },
  {
    "prompt": "What is the smallest unit of matter that retains the properties of an element?",
      "A": "Atom",
      "B": "Molecule",
      "C": "Proton",
      "D": "Electron",
      "E": "Neutron",
    "answer": "A"
  },
  {
    "prompt": "Which of the following is responsible for transmitting signals between nerve cells?",
      "A": "Hormones",
      "B": "Enzymes",
      "C": "Neurons",
      "D": "Red blood cells",
      "E": "Platelets",
    "answer": "C"
  },
  {
    "prompt": "What is the process by which plants convert sunlight into chemical energy?",
      "A": "Photosynthesis",
      "B": "Respiration",
      "C": "Transpiration",
      "D": "Fermentation",
      "E": "Digestion",
    "answer": "A"
  }
]
'''
df = pd.read_json(datajson)

/tmp/ipykernel_26/2598890986.py:50: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(datajson)


In [160]:
df

,prompt,A,B,C,D,E,answer
0,What is the Earth's primary source of energy?,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy,B
1,Which gas is most abundant in Earth's atmosphere?,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon,B
2,What is the smallest unit of matter that retai...,Atom,Molecule,Proton,Electron,Neutron,A
3,Which of the following is responsible for tran...,Hormones,Enzymes,Neurons,Red blood cells,Platelets,C
4,What is the process by which plants convert su...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion,A


In [118]:
trn = df.drop(columns=['answer'])

In [119]:
trn

,prompt,A,B,C,D,E
0,What is the Earth's primary source of energy?,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy
1,Which gas is most abundant in Earth's atmosphere?,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon
2,What is the smallest unit of matter that retai...,Atom,Molecule,Proton,Electron,Neutron
3,Which of the following is responsible for tran...,Hormones,Enzymes,Neurons,Red blood cells,Platelets
4,What is the process by which plants convert su...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion


In [120]:
model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half()

**Load data wikipedia indexFlat**

In [121]:
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")

In [122]:
prompt_embeddings = model.encode(sentences = trn.prompt.values, batch_size= BATCH_SIZE, show_progress_bar = True,convert_to_tensor=True, device=DEVICE , normalize_embeddings =True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [123]:
## Get the top 3 pages 
search_score, search_index = sentence_index.search(prompt_embeddings, 5)

In [124]:
## Save memory - delete sentence_index since it is no longer necessary
del sentence_index
del prompt_embeddings
_ = gc.collect()
libc.malloc_trim(0)

1

In [125]:
df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                     columns=['id', 'file'])

In [126]:
# df.loc[search_index[0]].copy()   lấy list row theo search_index to index df
print(search_index[0].shape)
x = df.loc[search_index[0]].copy()
print(x)

(5,)
               id       file
1446633    944638  e.parquet
1550966  23624536  e.parquet
1896164    113728  g.parquet
3109475  25283053  l.parquet
1550946   7635170  e.parquet


In [127]:
## Get the article and associated file location using the index
wikipedia_file_data = []

for i ,(scr,idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx # list id
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)

  0%|          | 0/5 [00:00<?, ?it/s]

In [128]:
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
print(wikipedia_file_data.shape)

(25, 3)


In [129]:
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)
del df
_ = gc.collect()
libc.malloc_trim(0)

1

In [130]:
wikipedia_file_data

,id,prompt_id,file
0,1430539,4,a.parquet
1,19334199,2,a.parquet
2,202898,1,a.parquet
3,202899,1,a.parquet
4,23092516,1,a.parquet
5,42445,2,d.parquet
6,23624536,0,e.parquet
7,7635170,0,e.parquet
8,944638,0,e.parquet
9,113728,0,g.parquet


In [131]:
wikipedia_file_data.file.unique()

array(['a.parquet', 'd.parquet', 'e.parquet', 'g.parquet', 'l.parquet',
       'm.parquet', 'n.parquet', 'p.parquet', 's.parquet'], dtype=object)

In [132]:
## Get the full text data
"""
    for i in list title wikipedia parquet(a,b,c,..z)
    _id - list id in title parquet (a/b/c/....)
    read parquet file in list and read [id,text] and 
    get all text in _id ||  _df_temp = _df[_df['id'].isin(_id)].copy()
    
    Giải phóng dữ liệu sau mỗi lần đọc file parquet
"""
wiki_text_data = []
for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])
    _df_temp = _df[_df['id'].isin(_id)].copy()
    del _df
    _ = gc.collect()
    libc.malloc_trim(0)
    wiki_text_data.append(_df_temp)
wiki_text_data =  pd.concat(wiki_text_data).reset_index(drop=True)
_ = gc.collect()

  0%|          | 0/9 [00:00<?, ?it/s]

In [133]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df




def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    
    offset 0 len(documents)   : Độ dài của văn bản

    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df
    
    
def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    
    bf.text_to_sentences_and_offsets(document)
    import blingfire as bf
    text_to_sentences_and_offsets  tách các câu đơn từ doccumente -> return list tuper(start,end)  
    
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    
    Đầu ra tách nhỏ list tuper(start,end )  trả về datafame row từng tupe in list 
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [134]:
processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values,disable_progress_bar = True)

In [135]:
## Get embeddings of the wiki text data
wiki_data_embeddings = model.encode(processed_wiki_text_data.text,
                                    batch_size=BATCH_SIZE,
                                    device=DEVICE,
                                    show_progress_bar=True,
                                    convert_to_tensor=True,
                                    normalize_embeddings=True)#.half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()
_ = gc.collect()

Batches:   0%|          | 0/143 [00:00<?, ?it/s]

In [136]:
## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)


## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']
# trn['prompt_answer_stem'] = trn['prompt']

In [137]:
question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
question_embeddings = question_embeddings.detach().cpu().numpy()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [138]:
## List containing just Context
contexts = []

for r in tqdm(trn.itertuples(), total=len(trn)):
    prompt_id = r.Index
    
    # list id_document  with prompt_id = index (len = 5) -> id  doccument
    list_file_data_with_prompt_id = wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values
    
    # đánh dấu row có id document in list_prompt_id
    list_text_in_document =  processed_wiki_text_data['document_id'].isin(list_file_data_with_prompt_id)
    # read data index with bool = True
    prompt_indices = processed_wiki_text_data[list_text_in_document].index.values
    if prompt_indices.shape[0] > 0: 
        # create store Flat
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        # read index embedding and add faiss store flat ->  bộ dữ liệu ngữ cảnh
        prompt_index.add(wiki_data_embeddings[prompt_indices])
        context = ""
        score, index = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(score[prompt_id], index[prompt_id]):
            context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
    contexts.append(context)   

  0%|          | 0/5 [00:00<?, ?it/s]

In [139]:
## List containing just Context
contexts = []

for r in tqdm(trn.itertuples(), total=len(trn)):
    prompt_id = r.Index
    
    # list id_document  with prompt_id = index (len = 5) -> id  doccument
    list_file_data_with_prompt_id = wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values
    
    # đánh dấu row có id document in list_prompt_id
    list_text_in_document =  processed_wiki_text_data['document_id'].isin(list_file_data_with_prompt_id)
    # read data index with bool = True
    prompt_indices = processed_wiki_text_data[list_text_in_document].index.values
    if prompt_indices.shape[0] > 0: 
        # create store Flat
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        # read index embedding and add faiss store flat ->  bộ dữ liệu ngữ cảnh
        prompt_index.add(wiki_data_embeddings[prompt_indices])
        context = ""
        score, index = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(score[prompt_id], index[prompt_id]):
            context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
    contexts.append(context)   

  0%|          | 0/5 [00:00<?, ?it/s]

In [140]:
trn['context'] = contexts

In [141]:
trn[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)

In [142]:
trn = trn[["prompt", "context", "A", "B", "C", "D", "E"]]

In [143]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoTokenizer
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union

In [144]:
MODEL_NAME = '/kaggle/input/weight-26-9-deberta-accuracy-0-88/checkpoints_1_microsoft_deberta_21_9'
path_test = '/kaggle/working/test_context.csv'
MAX_INPUT = 256

In [145]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [ "[CLS] " + example['context'] ] * 5
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first', 
                                  max_length=MAX_INPUT, add_special_tokens=False)
#     tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
#         label_name = 'label' if 'label' in features[0].keys() else 'labels'
#         labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
#         batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [146]:
df_test = trn

In [180]:
df_test

,prompt,context,A,B,C,D,E,prediction
0,What is the Earth's primary source of energy?,Electricity from nuclear power supplied 8% and...,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy,B
1,Which gas is most abundant in Earth's atmosphere?,The atmosphere of Earth is composed of nitroge...,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon,B
2,What is the smallest unit of matter that retai...,"For example, an atom of helium-4 has a mass of...",Atom,Molecule,Proton,Electron,Neutron,A
3,Which of the following is responsible for tran...,They send these signals in the form of electro...,Hormones,Enzymes,Neurons,Red blood cells,Platelets,C
4,What is the process by which plants convert su...,Photosynthesis can be described by the simplif...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion,A


In [148]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
dataset_test = Dataset.from_pandas(df_test)
tokenized_dataset_test = dataset_test.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E'])

In [151]:
model = AutoModelForMultipleChoice.from_pretrained(MODEL_NAME)
trainer = Trainer(model=model)

In [152]:
test_predictions = trainer.predict(tokenized_dataset_test).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]

In [153]:
predictions_as_answer_letters

array([['B', 'C', 'D', 'E', 'A'],
       ['B', 'C', 'A', 'D', 'E'],
       ['A', 'B', 'E', 'C', 'D'],
       ['C', 'D', 'A', 'E', 'B'],
       ['A', 'C', 'B', 'E', 'D']], dtype='<U1')

In [154]:
predictions_as_answer_letters[0]

array(['B', 'C', 'D', 'E', 'A'], dtype='<U1')

In [157]:
trn

,prompt,context,A,B,C,D,E,prediction
0,What is the Earth's primary source of energy?,Electricity from nuclear power supplied 8% and...,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy,B
1,Which gas is most abundant in Earth's atmosphere?,The atmosphere of Earth is composed of nitroge...,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon,B
2,What is the smallest unit of matter that retai...,"For example, an atom of helium-4 has a mass of...",Atom,Molecule,Proton,Electron,Neutron,A
3,Which of the following is responsible for tran...,They send these signals in the form of electro...,Hormones,Enzymes,Neurons,Red blood cells,Platelets,C
4,What is the process by which plants convert su...,Photosynthesis can be described by the simplif...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion,A


In [161]:
df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :1]
]

In [162]:
df

,prompt,A,B,C,D,E,answer,prediction
0,What is the Earth's primary source of energy?,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy,B,B
1,Which gas is most abundant in Earth's atmosphere?,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon,B,B
2,What is the smallest unit of matter that retai...,Atom,Molecule,Proton,Electron,Neutron,A,A
3,Which of the following is responsible for tran...,Hormones,Enzymes,Neurons,Red blood cells,Platelets,C,C
4,What is the process by which plants convert su...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion,A,A


****Dự đoán với ngữ cảnh được nhập vào****

In [170]:
json_gpt_context = '''
[
  {
    "context": "In the context of energy sources,",
    "prompt": "what is the Earth's primary source of energy?",
      "A": "Wind power",
      "B": "Solar radiation",
      "C": "Geothermal heat",
      "D": "Nuclear fission",
      "E": "Hydroelectric energy",
    "answer": "B"
  },
  {
    "context": "Considering atmospheric composition,",
    "prompt": "which gas is most abundant in Earth's atmosphere?",
      "A": "Carbon dioxide",
      "B": "Nitrogen",
      "C": "Oxygen",
      "D": "Methane",
      "E": "Argon",
    "answer": "B"
  },
  {
    "context": "In the study of matter,",
    "prompt": "what is the smallest unit of matter that retains the properties of an element?",
      "A": "Atom",
      "B": "Molecule",
      "C": "Proton",
      "D": "Electron",
      "E": "Neutron",
    "answer": "A"
  },
  {
    "context": "Regarding the nervous system,",
    "prompt": "which of the following is responsible for transmitting signals between nerve cells?",
      "A": "Hormones",
      "B": "Enzymes",
      "C": "Neurons",
      "D": "Red blood cells",
      "E": "Platelets",
    "answer": "C"
  },
  {
    "context": "In the process of photosynthesis,",
    "prompt": "what is the process by which plants convert sunlight into chemical energy?",
      "A": "Photosynthesis",
      "B": "Respiration",
      "C": "Transpiration",
      "D": "Fermentation",
      "E": "Digestion",
    "answer": "A"
  }
]
'''

In [181]:
df = pd.read_json(json_gpt_context)

/tmp/ipykernel_26/3391003687.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_gpt_context)


In [182]:
df

,context,prompt,A,B,C,D,E,answer
0,"In the context of energy sources,",what is the Earth's primary source of energy?,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy,B
1,"Considering atmospheric composition,",which gas is most abundant in Earth's atmosphere?,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon,B
2,"In the study of matter,",what is the smallest unit of matter that retai...,Atom,Molecule,Proton,Electron,Neutron,A
3,"Regarding the nervous system,",which of the following is responsible for tran...,Hormones,Enzymes,Neurons,Red blood cells,Platelets,C
4,"In the process of photosynthesis,",what is the process by which plants convert su...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion,A


In [177]:
df_test_context =  df.drop(columns=['answer'])

In [178]:
df_test_context

,context,prompt,A,B,C,D,E
0,"In the context of energy sources,",what is the Earth's primary source of energy?,Wind power,Solar radiation,Geothermal heat,Nuclear fission,Hydroelectric energy
1,"Considering atmospheric composition,",which gas is most abundant in Earth's atmosphere?,Carbon dioxide,Nitrogen,Oxygen,Methane,Argon
2,"In the study of matter,",what is the smallest unit of matter that retai...,Atom,Molecule,Proton,Electron,Neutron
3,"Regarding the nervous system,",which of the following is responsible for tran...,Hormones,Enzymes,Neurons,Red blood cells,Platelets
4,"In the process of photosynthesis,",what is the process by which plants convert su...,Photosynthesis,Respiration,Transpiration,Fermentation,Digestion


In [184]:
dataset_test = Dataset.from_pandas(df_test_context)
tokenized_dataset_test = dataset_test.map(preprocess, remove_columns=['prompt', 'context', 'A', 'B', 'C', 'D', 'E'])


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [185]:
tokenized_dataset_test

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5
})

In [ ]:
test_predictions = trainer.predict(tokenized_dataset_test).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
df_test_context['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :1]
]